In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline

In [2]:
DATA_DIR = "/wd/data/MNIST/"
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

Extracting ../data/MNIST/train-images-idx3-ubyte.gz
Extracting ../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
!ls -l ../data/MNIST/

total 379760
-rw-r--r--@ 1 abulbasar  staff   18289443 May  2 19:47 mnist_test.csv
-rw-r--r--@ 1 abulbasar  staff  109575994 May  2 19:48 mnist_train.csv
-rw-r--r--  1 abulbasar  staff    7840016 Apr  7 18:48 t10k-images-idx3-ubyte
-rw-r--r--  1 abulbasar  staff    1648877 Apr 30 12:49 t10k-images-idx3-ubyte.gz
-rw-r--r--  1 abulbasar  staff      10008 Apr  7 18:48 t10k-labels-idx1-ubyte
-rw-r--r--  1 abulbasar  staff       4542 Apr 30 12:49 t10k-labels-idx1-ubyte.gz
-rw-r--r--  1 abulbasar  staff   47040016 Apr  7 18:48 train-images-idx3-ubyte
-rw-r--r--  1 abulbasar  staff    9912422 Apr 30 12:49 train-images-idx3-ubyte.gz
-rw-r--r--  1 abulbasar  staff      60008 Apr  7 18:48 train-labels-idx1-ubyte
-rw-r--r--  1 abulbasar  staff      28881 Apr 30 12:49 train-labels-idx1-ubyte.gz


In [21]:
NUM_STEPS = 1000
MINIBATCH_SIZE = 100
learnin_rate = 0.2

x = tf.placeholder(tf.float32, [None, 784])
y_true = tf.placeholder(tf.float32, [None, 10])
print("Shape of x: ", x.get_shape())
print("Shape of y_true: ", y_true.get_shape())


W = tf.Variable(tf.zeros([784, 10]))
bias = tf.Variable(tf.zeros([10]))

y_pred = tf.matmul(x, W) + bias
print("Shape of y_pred: ", y_pred.get_shape())

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels= y_true, logits=y_pred))

train = tf.train.GradientDescentOptimizer(learnin_rate).minimize(cross_entropy)
#train = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

with tf.Session() as sess:

    # Train
    sess.run(tf.global_variables_initializer())

    for epoch in range(NUM_STEPS):
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        sess.run(train, feed_dict={x: batch_xs, y_true: batch_ys})

    # Test
    ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})
    

print("Accuracy: {:.4}%".format(ans*100))

Shape of x:  (?, 784)
Shape of y_true:  (?, 10)
Shape of y_pred:  (?, 10)
Accuracy: 91.5%


In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')


def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)


def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

#### weight_variable
These are the weights for either fully connected or convolutional layers of the network. They are initialized randomly using a truncated normal distribution with a standard deviation of .1. This sort of initialization with a random normal distribution that is truncated at the tails, is pretty common and generally produces good results (see note on random initialization below).

#### bias_variable 
The bias elements in either a fully connected or a convolutional layer. These are all initialized with the constant value of .1. 

#### conv2d 
The convolution we will typically use. A full convolution (no skips) with an output the same size as the input. 
max_pool_2x2 - max pool to half the size across the height/width dimensions, and in total quarter the size of the feature map.

#### conv_layer 
This is the actual layer we will use. Linear convolution as defined in conv2d, with a bias, and followed by the relu non-linearity.

#### full_layer 
A standard full layer with a bias. Notice that here we didn’t add the relu. This allows us to use the same layer for the final output where we don’t need the non-linear part.

In [6]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x_image, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)


In [7]:
mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels= y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(NUM_STEPS):
        batch = mnist.train.next_batch(50)

        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1],
                                                           keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))

        sess.run(train_step, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    X = mnist.test.images.reshape(10, 1000, 784)
    Y = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean([sess.run(accuracy, 
                             feed_dict={x:X[i], y_:Y[i],keep_prob:1.0}) 
                             for i in range(10)])

print("test accuracy: {}".format(test_accuracy))

Extracting ../data/MNIST/train-images-idx3-ubyte.gz
Extracting ../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.10000000149011612
step 100, training accuracy 0.8399999737739563
step 200, training accuracy 0.8600000143051147
step 300, training accuracy 0.8999999761581421
step 400, training accuracy 0.9399999976158142
step 500, training accuracy 0.8999999761581421
step 600, training accuracy 0.9399999976158142
step 700, training accuracy 0.9599999785423279
step 800, training accuracy 0.8799999952316284
step 900, training accuracy 0.9800000190734863
test accuracy: 0.95660001039505
